# Titanic Assignment

In [1]:
import pandas as pd 
import numpy as np 
from sklearn import tree
from sklearn import preprocessing
import seaborn as sns

In [2]:
titanic_train = pd.read_csv("train.csv")
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
titanic_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
titanic_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
titanic_train["Age"] = np.where(titanic_train["Age"].isnull(),28,titanic_train["Age"])
titanic_train["Embarked"] = np.where(titanic_train["Embarked"].isnull(),titanic_train["Embarked"].mode(),titanic_train["Embarked"])
print(titanic_train["Age"].isnull().sum())
print(titanic_train["Embarked"].isnull().sum())

0
0


In [6]:
label_encoder = preprocessing.LabelEncoder()
titanic_train["Sex"] = label_encoder.fit_transform(titanic_train["Sex"])
titanic_train["Embarked"] = label_encoder.fit_transform(titanic_train["Embarked"])
titanic_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,2


In [7]:
titanic_train["Sex"].isnull().sum()

0

# Important variable prediction using random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(n_estimators=1000,max_features=2,oob_score=True)
features=["Sex","Pclass","SibSp","Embarked","Age","Fare"]
rf_model.fit(X=titanic_train[features],y=titanic_train["Survived"])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [9]:
print("OOB Accurary")
print(rf_model.oob_score_)

OOB Accurary
0.8226711560044894


In [10]:
for feature,imp in zip(features,rf_model.feature_importances_):
    print(feature,imp)

Sex 0.2689713490939432
Pclass 0.08825388686470945
SibSp 0.05068366285757429
Embarked 0.03247636643851907
Age 0.27347447653769896
Fare 0.28614025820755506


In [14]:
tree_model=tree.DecisionTreeClassifier()
idv = pd.DataFrame([titanic_train["Sex"],titanic_train["Age"],titanic_train["Fare"]]).T
tree_model = tree.DecisionTreeClassifier(max_depth=17)
tree_model.fit(X=idv,y=titanic_train["Survived"])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=17, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [12]:
with open("Dtree6.dot","w")as f:
    f=tree.export_graphviz(tree_model,feature_names=["Sex","Age","Fare"],out_file=f)

In [13]:
tree_model.score(X=idv,y=titanic_train["Survived"])

0.9674523007856342